In [125]:
import cherrypy
import json
import redis
import uuid
from redis.commands.json.path import Path
from time import time

In [2]:
REDIS_HOST = 'redis-19577.c81.us-east-1-2.ec2.cloud.redislabs.com'
REDIS_PORT = 19577
REDIS_USERNAME = 'default'
REDIS_PASSWORD = 'so7BaeUMztM1pEpKusnIWrk9PN0nSVdm'

# Connect to Redis server
redis_client = redis.Redis(host=REDIS_HOST, port=REDIS_PORT, username=REDIS_USERNAME, password=REDIS_PASSWORD)
is_connected = redis_client.ping()
print('Redis Connected:', is_connected)

Redis Connected: True


In [16]:
# endpoint /status
class Status(object):
    exposed = True

    def GET(self, *path, **query):
        response_dict = {
            'status': 'online'
        }
        response = json.dumps(response_dict)

        return response

In [51]:
# endpoint /status
class Devices(object):
    exposed = True


    def GET(self, *path, **query):
        reg = '*:battery'
        keys = redis_client.keys('*:battery')
        items = set()

        for key in keys:
            key = key.decode()
            mac = key.removesuffix(':battery')
            items.add(mac)
        


        response_dict = {
            'mac_addresses': list(items)
        }
        response = json.dumps(response_dict)

        cherrypy.response.status = "200 OK: Everything worked as expected"

        return response

In [147]:
#endpoint /device
class Device(object):
    exposed = True

    def GET(self, *path, **query):

        #Check if the path is inserted
        if len(path) != 1:
            print('--------Missing path')
            #If not return a Bad request error
            raise cherrypy.HTTPError(400, message='Bad Request: missing MAC address.')

        mac_address = path[0]
        from_ = query.get('from', -1)
        to_ = query.get('to', -1)


        #Check if the start time is inserted
        if int(from_) < 0:
            #Return a Bad request error
            raise cherrypy.HTTPError(400, 'Bad Request: missing start time.')
        
        #Check if the end time is inserted
        if  int(to_) < 0:
            #Return a Bad request error
            raise cherrypy.HTTPError(400, 'Bad Request: missing end time.')

        
        battery_key = mac_address+':battery'
        power_key = mac_address+':power'
        print(type(battery_key))
        battery_values = redis_client.ts().range(battery_key, from_time=from_, to_time=to_)
        power_values = redis_client.ts().range(power_key, from_time=from_, to_time=to_)
        
        
        response_dict = {
            'mac_address': mac_address,
            'timestamps': list(map(lambda x: x[0], battery_values)),
            'battery_levels': list(map(lambda x: x[1], battery_values)),
            'power_plugged':list(map(lambda x:x[1], power_values))
        }
        response = json.dumps(response_dict)


        cherrypy.response.status = "200 OK: Everything worked as expected"
        return response

    def DELETE(self, *path, **query):
        if len(path) != 1:
            raise cherrypy.HTTPError(400, 'Bad Request: missing MAC address')
        mac_address = path[0]
        #Try to remove the item
        battery_key = mac_address+':battery'
        power_key = mac_address+':power'
        print(battery_key)
        battery_found = redis_client.delete(battery_key)
        power_found = redis_client.delete(power_key)

        #Check if the item has been removed
        if battery_found == 0 and power_found == 0:
            raise cherrypy.HTTPError(404, 'Not Found: invalid MAC address')

        return 



In [148]:
conf = {'/': {'request.dispatch': cherrypy.dispatch.MethodDispatcher()}}
cherrypy.tree.mount(Status(), '/online', conf)
cherrypy.tree.mount(Devices(), '/devices', conf)
cherrypy.tree.mount(Device(), '/device', conf)
cherrypy.config.update({'server.socket_host': '0.0.0.0'})
cherrypy.config.update({'server.socket_port': 8080})
cherrypy.engine.start()
cherrypy.engine.block()

[14/Jan/2023:16:57:22] ENGINE Bus STARTING
[14/Jan/2023:16:57:22] ENGINE Started monitor thread 'Autoreloader'.
[14/Jan/2023:16:57:22] ENGINE Serving on http://0.0.0.0:8080
[14/Jan/2023:16:57:22] ENGINE Bus STARTED


127.0.0.1 - - [14/Jan/2023:16:57:25] "GET /devices HTTP/1.1" 200 91 "" "vscode-restclient"
127.0.0.1 - - [14/Jan/2023:16:57:26] "GET /online HTTP/1.1" 200 20 "" "vscode-restclient"
<class 'str'>
127.0.0.1 - - [14/Jan/2023:16:57:28] "GET /device/0xdca9047270e2?from=0&to=1673710261095 HTTP/1.1" 200 94 "" "vscode-restclient"
0xdca9047270e2:battery
127.0.0.1 - - [14/Jan/2023:16:57:31] "DELETE /device/0xdca9047270e2 HTTP/1.1" 200 - "" "vscode-restclient"
127.0.0.1 - - [14/Jan/2023:16:57:33] "GET /devices HTTP/1.1" 200 73 "" "vscode-restclient"
127.0.0.1 - - [14/Jan/2023:16:57:56] "GET /devices HTTP/1.1" 200 73 "" "vscode-restclient"
127.0.0.1 - - [14/Jan/2023:16:58:00] "GET /online HTTP/1.1" 200 20 "" "vscode-restclient"
127.0.0.1 - - [14/Jan/2023:16:58:02] "GET /devices HTTP/1.1" 200 73 "" "vscode-restclient"
<class 'str'>
127.0.0.1 - - [14/Jan/2023:16:58:07] "GET /device/0x17a9ed6a3019?from=0&to=1673710261095 HTTP/1.1" 200 660 "" "vscode-restclient"
0x17a9ed6a3019:battery
127.0.0.1 - - [1

[14/Jan/2023:16:58:24] ENGINE Keyboard Interrupt: shutting down bus
[14/Jan/2023:16:58:24] ENGINE Bus STOPPING
[14/Jan/2023:16:58:25] ENGINE HTTP Server cherrypy._cpwsgi_server.CPWSGIServer(('0.0.0.0', 8080)) shut down
[14/Jan/2023:16:58:25] ENGINE Stopped thread 'Autoreloader'.
[14/Jan/2023:16:58:25] ENGINE Bus STOPPED
[14/Jan/2023:16:58:25] ENGINE Bus EXITING
[14/Jan/2023:16:58:25] ENGINE Bus EXITED
[14/Jan/2023:16:58:25] ENGINE Waiting for child threads to terminate...
